References:
- https://twitter.com/karpathy/status/1647025230546886658
- https://github.com/karpathy/randomfun/blob/master/knn_vs_svm.ipynb
- https://www.cs.cmu.edu/~tmalisie/projects/iccv11/

In [1]:
import numpy as np
import sklearn
from sklearn import svm

SEED = 42
np.random.seed(SEED)

In [2]:
print('NumPy version:', np.__version__)
print('scikit-learn version:', sklearn.__version__)

NumPy version: 1.22.4
scikit-learn version: 1.2.2


## Embedding Matrix and Query Vector

In [3]:
N_DOCUMENTS = 1000
EMBEDDING_DIM = 1536

In [4]:
# create a matrix of size: N_DOCUMENTS x EMBEDDING_DIM
embeddings_matrix = np.random.randn(N_DOCUMENTS, EMBEDDING_DIM) 

# L2 normalize the rows, as is common
embeddings_matrix = embeddings_matrix / \
                    np.sqrt((embeddings_matrix**2).sum(1, keepdims=True)) 

print(embeddings_matrix.shape)
print(embeddings_matrix)

(1000, 1536)
[[ 0.01278695 -0.00355935  0.0166735  ... -0.00379975 -0.01199722
  -0.04105258]
 [ 0.01330037 -0.01379501 -0.03029659 ...  0.017939   -0.03287104
   0.04408893]
 [ 0.00512884  0.04136598 -0.01858167 ... -0.00643044 -0.00874959
   0.01240212]
 ...
 [-0.01519871 -0.01918601 -0.00795636 ...  0.05916144  0.06012371
  -0.00636577]
 [ 0.04927123  0.02736768  0.02532746 ...  0.01941154 -0.00600695
  -0.03203716]
 [-0.00810883 -0.00925525  0.00068903 ...  0.02962341  0.01253986
  -0.02029635]]


In [5]:
# the query vector of size: EMBEDDING_DIM x 1
query_vector = np.random.randn(EMBEDDING_DIM)

# L2 normalization
query_vector = query_vector / np.sqrt((query_vector**2).sum())
query_vector

array([ 0.00802579,  0.02001674, -0.03141846, ...,  0.01385069,
       -0.02867884, -0.02033733])

## KNN vs SVM

A very common workflow is to index some data based on its embeddings and then given a new query embedding retrieve the most similar examples with k-Nearest Neighbor search. For example, you can imagine embedding a large collection of papers by their abstracts and then given a new paper of interest retrieve the most similar papers to it.

TLDR in my experience it ~always works better to use an SVM instead of kNN, if you can afford the slight computational hit. Example below:

### KNN

In [6]:
# Tired: use KNN

# (N_DOCUMENTS x EMBEDDING_DIM) x (EMBEDDING_DIM x 1) 
# result is of size: N_DOCUMENTS x 1
similarities = embeddings_matrix.dot(query_vector)

# view the first 5 rows
similarities[:5]

array([-0.00591654, -0.00823215,  0.01871934,  0.01213897,  0.00368823])

In [7]:
# sort the results
sorted_ix = np.argsort(-similarities)

print("Top 10 results:")
for k in sorted_ix[:10]:
  print(f"Row {k}, Similarity {similarities[k]}")

Top 10 results:
Row 545, Similarity 0.07956628031855814
Row 790, Similarity 0.07109372365891174
Row 973, Similarity 0.06920799481214632
Row 597, Similarity 0.06474824575503951
Row 479, Similarity 0.06350781255023313
Row 229, Similarity 0.0614321834997024
Row 976, Similarity 0.061222853526241586
Row 568, Similarity 0.060888722805113274
Row 800, Similarity 0.06007081261453451
Row 654, Similarity 0.058158824328240384


### SVM

In [8]:
# Wired: use an SVM

# create the "Dataset"

# x is (N_DOCUMENTS + 1, EMBEDDING_DIM) matrix, with query now as the first row
X = np.concatenate([query_vector[None, ...], embeddings_matrix])
print(X.shape)
print(query_vector)
print(X)

(1001, 1536)
[ 0.00802579  0.02001674 -0.03141846 ...  0.01385069 -0.02867884
 -0.02033733]
[[ 0.00802579  0.02001674 -0.03141846 ...  0.01385069 -0.02867884
  -0.02033733]
 [ 0.01278695 -0.00355935  0.0166735  ... -0.00379975 -0.01199722
  -0.04105258]
 [ 0.01330037 -0.01379501 -0.03029659 ...  0.017939   -0.03287104
   0.04408893]
 ...
 [-0.01519871 -0.01918601 -0.00795636 ...  0.05916144  0.06012371
  -0.00636577]
 [ 0.04927123  0.02736768  0.02532746 ...  0.01941154 -0.00600695
  -0.03203716]
 [-0.00810883 -0.00925525  0.00068903 ...  0.02962341  0.01253986
  -0.02029635]]


In [9]:
# y is (N_DOCUMENTS + 1, 1) array
y = np.zeros(N_DOCUMENTS + 1)
y[0] = 1 # we have a single positive example, mark it as such
print(y.shape)
print(y)

(1001,)
[1. 0. 0. ... 0. 0. 0.]


In [10]:
# train our (Exemplar) SVM
# https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html
linear_svc_model = svm.LinearSVC(
    class_weight='balanced', 
    verbose=False, 
    max_iter=10000, 
    tol=1e-6, # tolerance for stopping criteria
    C=0.1, # regularization parameter
).fit(X, y) # train

linear_svc_model

LinearSVC(C=0.1, class_weight='balanced', max_iter=10000, tol=1e-06,
          verbose=False)

In [11]:
# infer on whatever data you wish, e.g. the original data
similarities = linear_svc_model.decision_function(X)
similarities

array([ 0.97971126, -0.98920288, -0.99426439, ..., -1.06692443,
       -0.95013296, -0.99316836])

In [12]:
sorted_ix = np.argsort(-similarities)

print("Top 10 results:")
for k in sorted_ix[:10]:
  print(f"Row {k}, Similarity {similarities[k]}")

Top 10 results:
Row 0, Similarity 0.9797112617216351
Row 546, Similarity -0.8360649738915675
Row 791, Similarity -0.8519226181122038
Row 974, Similarity -0.8585435504683989
Row 480, Similarity -0.8620392370633861
Row 598, Similarity -0.8653315003700203
Row 230, Similarity -0.8671983886478062
Row 569, Similarity -0.8674761579346135
Row 977, Similarity -0.8705646065664832
Row 801, Similarity -0.8728033782558365


In practice you will find that this ordering:

- is of higher quality
- is slower: we have to train an SVM
- can easily accommodate a number of positives not just one, so it is more flexible
- don't be scared of having a single positive and everything else being negative. this is totally fine!
- if you have way way too many negatives, consider subsampling and only using a portion of them.


**Value of C**: You'll want to tune C. You'll most likely find the best setting to be between 0.01 and 10. Values like 10 very severely penalize the classifier for any mispredictions on your data. It will make sure to fit your data. Values like 0.01 will incur less penalty and will be more regularized. Usually this is what you want. I find that in practice a value like 0.1 works well if you only have a few examples that you don't trust too much. If you have more examples and they are very noise-free, try more like 1.0


**Why does this work?** In simple terms, because SVM considers the entire cloud of data as it optimizes for the hyperplane that "pulls apart" your positives from negatives. In comparison, the kNN approach doesn't consider the global manifold structure of your entire dataset and "values" every dimension equally. The SVM basically finds the way that your positive example is unique in the dataset, and then only considers its unique qualities when ranking all the other examples.

## Dependencies

In [13]:
!pip install session-info

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
import session_info

session_info.show()